<h1 align = "center"><b> XGBoost </b></h1>

&emsp;&emsp; Nesta etapa, vamos analisar e transformar os dados para que possamos aplicar o modelo XGBoost.

In [350]:
import pandas as pd
import numpy as np
import holidays
import matplotlib.pyplot as plt

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [351]:
df = pd.read_csv('../Demanda Diária.csv', index_col = 0, parse_dates = True)
df = df.loc[df['loja'] == 1]
df = df.loc[df['produto'] == 1]
df = df.drop(['loja', 'produto'], axis = 1)

<h2 align = "center"><b> Desmembrando a Data </b></h2>

In [352]:
def DesmembrarData (df):
    df['Dia'] = df.index.day
    df['Mês'] = df.index.month
    df['Ano'] = df.index.year
    df['Dia_da_Semana'] = df.index.dayofweek # Domingo = 1
    df['Dia_do_Ano'] = df.index.dayofyear
    df['Fim_de_Semana'] = df['Dia_da_Semana'].map(lambda x: True if x >= 5 else False)
    df['start_of_month'] = df.index.is_month_start.astype(int)
    df['end_of_month'] = df.index.is_month_end.astype(int)

    br_holidays = holidays.Brazil()
    df['Feriado'] = df.index.map(lambda x: True if x in br_holidays else False)

    df = pd.get_dummies(df, columns = ['Dia', 'Mês', 'Dia_da_Semana'])     
    df = df.rename(columns={'Dia_da_Semana_0': 'Segunda', 'Dia_da_Semana_1': 'Terça', 'Dia_da_Semana_2': 'Quarta', 'Dia_da_Semana_3': 'Quinta', 'Dia_da_Semana_4': 'Sexta', 'Dia_da_Semana_5': 'Sábado', 'Dia_da_Semana_6': 'Domingo'})
    
    return df

In [353]:
df = DesmembrarData(df)

<h2 align = "center"><b> Desmembrando o Número de Vendas </b></h2>

In [354]:
def CriarLags (df):
    Lista_Lags = [1, 2, 3, 4, 5, 6, 7, 28, 29, 30, 31, 61, 122, 183, 365] # Ultima semana, 1 mês, 2 meses, 3 meses, 6 meses, 1 ano, 2 anos
    for i in Lista_Lags:
        df['Lag_' + str(i)] = df['itens_vendidos'].shift(i) # Lag = Atraso, é o valor da variável em um determinado período de tempo anterior
    return df

def CriarMédiasMóveis (df): # Usando Lag_1 para Evitar Vazamento de Dados
    Janelas = [3, 4, 5, 6, 7, 28, 29, 30, 31, 61, 122, 183, 365]  
    for i in Janelas: df['Media_Movel_' + str(i)] = df['Lag_1'].rolling(i).mean()
    for i in Janelas: df['EWM_' + str(i)] = df['Lag_1'].ewm(span = i).mean() # Média Móvel Ponderada Exponencialmente
    return df

In [355]:
df = CriarLags(df)
df = CriarMédiasMóveis(df)

<h2 align = "center"><b> Criando o Modelo XGBoost </b></h2>

In [356]:
x_treino = df.iloc[:-31].drop('itens_vendidos', axis = 1)
x_teste = df.iloc[-31:].drop('itens_vendidos', axis = 1)
y_treino = df.iloc[:-31]['itens_vendidos']
y_teste = df.iloc[-31:]['itens_vendidos']

tam_val = len(x_treino) * 0.2
x_val = x_treino[-int(tam_val):]
y_val = y_treino[-int(tam_val):]
x_treino = x_treino[:-int(tam_val)]
y_treino = y_treino[:-int(tam_val)]

In [357]:
import xgboost as xgb
modelo = xgb.XGBRegressor()

In [358]:
modelo.fit(x_treino, y_treino, eval_set = [(x_val, y_val)], verbose = True, eval_metric = 'rmse', early_stopping_rounds = 20)

[0]	validation_0-rmse:6.56068
[1]	validation_0-rmse:5.92052
[2]	validation_0-rmse:5.52610
[3]	validation_0-rmse:5.33006
[4]	validation_0-rmse:5.28278
[5]	validation_0-rmse:5.21523
[6]	validation_0-rmse:5.17751


`eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.


[7]	validation_0-rmse:5.12941
[8]	validation_0-rmse:5.12256
[9]	validation_0-rmse:5.14737
[10]	validation_0-rmse:5.16664
[11]	validation_0-rmse:5.19360
[12]	validation_0-rmse:5.21887
[13]	validation_0-rmse:5.23734
[14]	validation_0-rmse:5.23341
[15]	validation_0-rmse:5.23937
[16]	validation_0-rmse:5.24920
[17]	validation_0-rmse:5.25217
[18]	validation_0-rmse:5.26667
[19]	validation_0-rmse:5.27117
[20]	validation_0-rmse:5.28358
[21]	validation_0-rmse:5.30677
[22]	validation_0-rmse:5.30271
[23]	validation_0-rmse:5.30451
[24]	validation_0-rmse:5.31497
[25]	validation_0-rmse:5.31168
[26]	validation_0-rmse:5.31949
[27]	validation_0-rmse:5.33002


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [359]:
previsoes = modelo.predict(x_teste)
previsoes = pd.DataFrame(previsoes, index = x_teste.index)

In [360]:
resultado = pd.concat([y_teste, previsoes], axis = 1)
resultado.columns = ['Valor Real', 'Previsão']

In [362]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
print('RMSE: ', mean_squared_error(resultado['Valor Real'], resultado['Previsão'], squared = False))
print('MAE: ', mean_absolute_error(resultado['Valor Real'], resultado['Previsão']))

RMSE:  4.980819486922266
MAE:  3.876465243677939


In [ ]:
plt.plot(resultado['Valor Real'], label = 'Valor Real')
plt.plot(resultado['Previsão'], label = 'Previsão')
plt.legend(loc = 'best')
plt.title('Previsão XGBoost')
plt.show()

<img src="./Gráficos/Previsão%20XGBoost.png" alt="Previsão XGBoost" width="100%">


<h2 align = "center"><b> Aprendendo com o Modelo XGBoost </b></h2>

&emsp;&emsp; Nesta fase, vamos utilizar o SHAP para aprofundar nossa compreensão sobre como o modelo XGBoost está absorvendo informações dos dados. Essa abordagem é benéfica, pois, além de oferecer insights sobre o comportamento do modelo, proporciona maior confiança nos resultados, permitindo identificar quais variáveis desempenham um papel mais crucial na tomada de decisões.

In [ ]:
import shap # Dependendo do gráfico pode ser necessário executar: shap.initjs() 
explainer = shap.Explainer(modelo)

In [344]:
shap_values = explainer(x_treino)

In [345]:
# Cria uma ordem para as variáveis
col2num = {col: i for i, col in enumerate(x_treino.columns)}
order = list(map(col2num.get, x_treino.columns))

In [ ]:
shap.plots.beeswarm(shap_values, max_display = 200, order = order)

<img src="./Gráficos/Beeswarm.png" alt="Beeswarm" width="100%">

`Sem impacto evidente:` Ano, start_of_month, end_of_month

`Dia do Ano:` Aparentemente, após o meio do ano, as vendas diminuem. Esses valores estão com impacto negativo na previsão do modelo.

`Fim de Semana:` Sem sombra de dúvidas é um fator importante. Aparentemente, as vendas são maiores nos finais de semana.

`Feriado:` Aparentemente, as vendas são maiores nos feriados. Porém, o impacto é pequeno.

`Dia do Mês:` Visualmente, o dia do mês impacta nas vendas (cada um com um peso diferente).

`Meses:` Como ocorre com o dia do mês, cada mês tem um impacto diferente nas vendas.

`Dias da Semana:` De segunda a quinta as vendas são menores, sendo segunda o dia com maior impacto negativo. Sexta, sábado e domingo as vendas são maiores, sendo sexta o dia com maior impacto positivo.

`Lag:` Visualmente, quanto maior o número de vendas nos dias anteriores, maior o número de vendas no dia atual (quando o valor é acima da média impacta positivamente, quando é abaixo impacta negativamente). O único que não segue essa regra é o Lag_122, que tem um impacto contrário ao esperado.

`Média Móvel:` Aparentemente, a média segue a mesma regra do Lag.

`EWM:` Aparentemente, segue a mesma regra do lag e da média móvel. Porém, o impacto é menor.

In [ ]:
shap.plots.waterfall(shap_values[-1], max_display = 20)

In [ ]:
shap.plots.scatter(shap_values[:,'Dia_do_Ano'])

<img src="./Gráficos/Scatter - Dia do Ano.png" alt="Scatter - Dia do Ano" width="100%">

In [ ]:
shap.plots.scatter(shap_values[:,'Feriado'], color=shap_values[:,'Dia_do_Ano'])

<img src="./Gráficos/Scatter - Feriado.png" alt="Scatter - Feriado" width="100%">